# Carbon Capture

## Chemical Reaction ( Part 1 Carbamic Acid )

$$NH_3 + CO_2 \rightarrow NH_{2}COOH$$

first of all we want to model the vibrational ground state enegies of the reactants using VQE

In [1]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver

driver = PySCFDriver(
    atom="N 0 0 0.111100; H 0 0.931600 -0.259200; H 0.806800 -0.465800 -0.259200; H -0.806800 -0.465800 -0.259200",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

es_problem = driver.run()

In [2]:
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer

transformer = ActiveSpaceTransformer(
    num_electrons=4,
    num_spatial_orbitals=4
)
es_problem = transformer.transform(es_problem)

In [3]:
from qiskit_nature.second_q.mappers import JordanWignerMapper

mapper = JordanWignerMapper()

In [4]:
from qiskit_algorithms import NumPyMinimumEigensolver

numpy_solver = NumPyMinimumEigensolver()

In [5]:
from qiskit_algorithms import VQE
from qiskit.primitives import StatevectorEstimator
from qiskit_algorithms.optimizers import SLSQP
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD


ansatz = UCCSD(
    es_problem.num_spatial_orbitals,
    es_problem.num_particles,
    mapper,
    initial_state=HartreeFock(
        es_problem.num_spatial_orbitals,
        es_problem.num_particles,
        mapper,
    ),
)

vqe_solver = VQE(StatevectorEstimator(), ansatz, SLSQP())
vqe_solver.initial_point = [0.0] * ansatz.num_parameters

In [6]:
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

calc = GroundStateEigensolver(mapper, vqe_solver)

In [7]:
res = calc.solve(es_problem)
print(res)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -67.526098692891
  - computed part:      -5.3504223429
  - ActiveSpaceTransformer extracted energy part: -62.175676349991
~ Nuclear repulsion energy (Hartree): 12.068827248365
> Total ground state energy (Hartree): -55.457271444525
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 4.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  0.00018897]
 
  0: 
  * Electronic dipole moment (a.u.): [-0.000001653821  -0.009505898705  0.692440730371]
    - computed part:      [-0.000001653888  0.79843278527  0.859346573184]
    - ActiveSpaceTransformer extracted energy part: [0.000000000067  -0.807938683975  -0.166905842812]
  > Dipole moment (a.u.): [0.000001653821  0.009505898705  -0.692251760371]  Total: 0.692317024094
                 (debye): [0.000004203593  0.024161582206  -1.759528302865]  Total: 1.759694186685
 
